# Feature Based Opinion Mining

## Topic - Apple Iphone 12

### Importing the Libraries

In [1]:
pip install requests_html

  Using cached requests_html-0.10.0-py3-none-any.whl (13 kB)
  Using cached parse-1.19.0.tar.gz (30 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pyppeteer-1.0.2-py3-none-any.whl (83 kB)
     ---------------------------------------- 62.8/62.8 kB 3.3 MB/s eta 0:00:00
  Using cached pyquery-2.0.0-py3-none-any.whl (22 kB)
  Using cached fake_useragent-1.1.1-py3-none-any.whl (50 kB)
  Using cached websockets-10.4-cp39-cp39-win_amd64.whl (101 kB)
     -------------------------------------- 140.6/140.6 kB 4.2 MB/s eta 0:00:00
  Using cached pyee-8.2.2-py2.py3-none-any.whl (12 kB)
     ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00
  Using cached importlib_resources-5.10.2-py3-none-any.whl (34 kB)
  Using cached cssselect-1.2.0-py2.py3-none-any.whl (1

In [2]:
pip install contractions

  Using cached contractions-0.1.73-py2.py3-none-any.whl (8.7 kB)
  Using cached textsearch-0.0.24-py2.py3-none-any.whl (7.6 kB)
  Using cached anyascii-0.3.1-py3-none-any.whl (287 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install nltk

     ---------------------------------------- 1.5/1.5 MB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB ? eta 0:00:00
     -------------------------------------- 267.8/267.8 kB 3.3 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install mlxtend

  Using cached mlxtend-0.21.0-py2.py3-none-any.whl (1.3 MB)
     ---------------------------------------- 7.2/7.2 MB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 42.5/42.5 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 8.3/8.3 MB 1.3 MB/s eta 0:00:00
     -------------------------------------- 160.2/160.2 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 965.4/965.4 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 751.9 kB/s eta 0:00:00
     -------------------------------------- 55.4/55.4 kB 289.6 kB/s eta 0:00:00
     -------------------------------------- 98.3/98.3 kB 297.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
# maintianing the Session to scrape the reviews
from requests_html import HTMLSession

# For preprocessing the scrape reviews
import re
import contractions
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import word_tokenize


# For finding the frequent patterns in the reviews
from mlxtend.frequent_patterns import apriori, association_rules

# Used for Progress Bar
from time import sleep
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [8]:
# Helper Methods
def getting_review_div(url, session, header, cookies):
    temp = session.get(url, headers = header, cookies = cookies)
    if not temp.html.find('div[data-hook=review]'):
        return False
    else:
        return temp.html.find('div[data-hook=review]')
    
def getting_reviews(asin, page_no):
    session = HTMLSession()
    url = f"https://www.amazon.in/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber={page_no}"
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
    cookies = {'session-id':'130-2350219-8238762'}
    reviews = getting_review_div(url, session, header, cookies)
    total = []

    for review in reviews:
        title = review.find('a[data-hook=review-title]', first = True).text
        body = review.find('span[data-hook=review-body]', first = True).text
        rating = review.find('i[data-hook=review-star-rating]', first = True).text
        data = {'Title':title, 'Reviews':body, 'Rating':rating}
        total.append(data)
    return total

def preprocessing(df):
    
    for i in range(len(df.Reviews)):
        stripped_text = contractions.fix(df.Reviews[i])
        df.Reviews[i] = stripped_text
    
        # Converting Reviews to Lower Case
        df.Reviews[i] = df.Reviews[i].lower()
    
        # Removing Punctuations
        df.Reviews[i] = re.sub("[^A-Za-z0-9]+", " ", df.Reviews[i])
    
        # Removing extra white spaces
        df.Reviews[i] = df.Reviews[i].replace("  ", " ").strip()
    
        # Lemmatizing 
        lem = WordNetLemmatizer()
        df.Reviews[i] = " ".join([lem.lemmatize(w) for w in df.Reviews[i].split()])
        
    return df

In [9]:
df = pd.DataFrame(columns = ['Title', 'Reviews', 'Rating'])
asin = "B08L5TNJHG"

for page in tqdm(range(1, 200)):
    temp = getting_reviews(asin = asin, page_no = str(page))
    df = df.append(temp, ignore_index=True)
    sleep(0.5)

100%|██████████| 199/199 [04:29<00:00,  1.35s/it]


In [10]:
df

,Title,Reviews,Rating
0,Nice iPhone,Received parcel in good condition. Happy using it,4.0 out of 5 stars
1,Very bad product,Bad products,1.0 out of 5 stars
2,Badiya he phone,,5.0 out of 5 stars
3,Slow in charging fast battery drawing,Not happy with battery life and want to exchange,2.0 out of 5 stars
4,tagada selfie maarta Hai Shukla ji,I like the camera quality 👌\nAnd good Battery ...,4.0 out of 5 stars
...,...,...,...
1985,"Best camera, Best & fast iOS system, Simple op...",The A14-Bionic chip makes it fast operating de...,5.0 out of 5 stars
1986,Heating while charging,Other than that everything is 👍.,4.0 out of 5 stars
1987,Very nice and cool colour of this phone and de...,It works good and u can for it. This iPhone ha...,5.0 out of 5 stars
1988,That it is original product,Good products,5.0 out of 5 stars


In [6]:
pip install wordnet

Note: you may need to restart the kernel to use updated packages.


In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pooji\AppData\Roaming\nltk_data...


True

In [10]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\pooji\AppData\Roaming\nltk_data...


True

In [11]:
df = preprocessing(df)
df

,Title,Reviews,Rating
0,Nice iPhone,received parcel in good condition happy using it,4.0 out of 5 stars
1,Very bad product,bad product,1.0 out of 5 stars
2,Badiya he phone,,5.0 out of 5 stars
3,Slow in charging fast battery drawing,not happy with battery life and want to exchange,2.0 out of 5 stars
4,tagada selfie maarta Hai Shukla ji,i like the camera quality and good battery bac...,4.0 out of 5 stars
...,...,...,...
1985,"Best camera, Best & fast iOS system, Simple op...",the a14 bionic chip make it fast operating dev...,5.0 out of 5 stars
1986,Heating while charging,other than that everything is,4.0 out of 5 stars
1987,Very nice and cool colour of this phone and de...,it work good and you can for it this iphone ha...,5.0 out of 5 stars
1988,That it is original product,good product,5.0 out of 5 stars


    Now, we are ready with the data frame which contain the reviews, rating and title.

### POS Tagging

In [12]:
#Save the NOUNS only
nouns = []
for i in range(len(df.Reviews)):
    pair = nltk.pos_tag(word_tokenize(df.Reviews[i]))
    for j in pair:
        if ('NN') in j[1] and len(j[0]) >3:
            nouns.append(j[0])
            
nouns = [*set(nouns)]

In [13]:
dic = {}
for i in tqdm(range(len(nouns))):
    temp = []
    for j in range(len(df.Reviews)):
        if nouns[i] in df.Reviews[j]:
            temp.append(1)
        else:
            temp.append(0)
        dic[nouns[i]] = temp
    sleep(0.2)

100%|██████████| 1416/1416 [05:14<00:00,  4.50it/s]


In [14]:
data = pd.DataFrame(dic)

In [15]:
apr_df = apriori(data, min_support = 0.03, use_colnames = True, max_len=2, verbose = 0)

### Compactness Pruning

In [16]:
dic = {}
len_list = [tuple(i) for i in apr_df.itemsets if len(i)>=2]
for feature in len_list:
    temp = 0
    for rev in df.Reviews:
        rev = list(rev.split())
        if feature[0] in rev and feature[1] in rev:
            i1 = rev.index(feature[0])
            i2 = rev.index(feature[1])
            if abs(i1-i2)<=3:
                temp = temp + 1
                dic[feature] = temp
                
compact_features = []
for k,v in dic.items():
    if v > 2:
        compact_features.append((k,v))
compact_features

[(('back', 'phone'), 3),
 (('battery', 'back'), 3),
 (('issue', 'phone'), 6),
 (('work', 'phone'), 3),
 (('amazon', 'iphone'), 6),
 (('amazon', 'phone'), 14),
 (('amazon', 'product'), 18),
 (('iphone', 'apple'), 9),
 (('apple', 'phone'), 11),
 (('product', 'apple'), 24),
 (('awesome', 'phone'), 14),
 (('experience', 'phone'), 6),
 (('iphone', 'quality'), 12),
 (('camera', 'iphone'), 11),
 (('good', 'iphone'), 15),
 (('iphone', 'phone'), 10),
 (('iphone', 'product'), 7),
 (('iphone', 'battery'), 3),
 (('camera', 'quality'), 87),
 (('good', 'quality'), 33),
 (('quality', 'phone'), 10),
 (('battery', 'quality'), 17),
 (('camera', 'life'), 12),
 (('life', 'good'), 25),
 (('life', 'phone'), 11),
 (('life', 'battery'), 125),
 (('android', 'phone'), 13),
 (('camera', 'good'), 43),
 (('camera', 'phone'), 18),
 (('camera', 'battery'), 46),
 (('backup', 'battery'), 55),
 (('time', 'phone'), 3),
 (('good', 'phone'), 66),
 (('good', 'product'), 48),
 (('good', 'battery'), 54),
 (('product', 'phone

    These are the most frequent features found after compactness pruning.

### Redundancy Pruning

In [17]:
single = [tuple(i) for i in apr_df.itemsets if len(i)==1]

In [18]:
double = [tuple(i) for i in apr_df.itemsets if len(i)==2]

In [19]:
total_dic_single = {}
for item in single:
    count = 0
    for rev in df.Reviews:
        rev = list(rev.split())
        if item[0] in rev:
            count = count + 1
    total_dic_single[item[0]] = count

In [20]:
total_dic_double = {}
for item in double:
    count = 0
    for rev in df.Reviews:
        rev = list(rev.split())
        if item[0] in rev and item[1] in rev:
            count = count + 1
    total_dic_double[(item[0],item[1])] = count

In [21]:
for item in single:
    for rev in df.Reviews:
        count = 0
        rev = list(rev.split())
        for feature in double:
            if item[0] in feature and feature[0] in rev and feature[1] in rev:
                count += 1
                total_dic_single[item[0]] = total_dic_single[item[0]] - count
                break

In [22]:
total_dic_single

{'back': 13,
 'working': 69,
 'charge': 24,
 'love': 56,
 'thank': 29,
 'amazo': 1,
 'play': 6,
 'smooth': 56,
 'issue': 53,
 'form': 2,
 'user': 68,
 'work': 25,
 'fast': 60,
 'mobile': 86,
 'nice': 131,
 'amazon': 29,
 'apple': 42,
 'worth': 57,
 'awesome': 71,
 'super': 32,
 'experience': 47,
 'screen': 63,
 'iphone': 75,
 'money': 89,
 'phon': 0,
 'quality': 17,
 'life': 1,
 'android': 22,
 'performance': 95,
 'camera': 37,
 'price': 85,
 'mobil': 1,
 'backup': 1,
 'cell': 2,
 'time': 38,
 'overall': 63,
 'excellent': 64,
 'good': 116,
 'heat': 22,
 'phone': 127,
 'hone': 0,
 'issu': 0,
 'batter': 0,
 'display': 60,
 'everything': 81,
 'product': 104,
 'medium': 136,
 'feature': 67,
 'purchase': 41,
 'feel': 62,
 'look': 56,
 'thing': 11,
 'delivery': 71,
 'battery': 36,
 'month': 53,
 'amazing': 90,
 'face': 62}

### Level of Automation

    With the help of this framework we can get all the important features from the reviews which are giving some 
    support towards buying the product.
    In this we just have to change the asin of the product which is nothing but the unique identification number 
    given by amazon for the product.

### Advantages

    Using this framework we can spead up the process of finding the features based on the opinions of reviewers.
    Which in turn will help to give the quick results.

### Disadvantages

    This framework is computationally expensive as creating the Binary Matrix from nouns take a lot of time.